## Setup autoreload, warnings and helper functions

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, Markdown
def print_heading(string):
    display(Markdown(f"# {string}"))
def print_subheading(string):
    display(Markdown(f"## {string}"))

## Set the visibility of cuda devices (in case your system contains more than one)

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

## Imports

In [ ]:
%matplotlib inline
from config import config
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import zipfile
import tensorflow as tf
import wget
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from torchvision.datasets.utils import download_url
from datasets import COCOImageDataset, DatasetSlice

from tools import apply_pipeline, crop_step, curry, apply_pipeline_cocodataset, get_save_step
from tonemapping.tonemapping import tonemap, tonemap_step
from segmentation.segmentation import segment
from pattern_extraction.extract_pattern import extract_pattern
from reidentification.identify import encode_single, encode_pipeline, create_database, identify, identify_single
from reidentification.visualisation import visualise_match
from simple_database import SimpleDatabase

## Create a configuration file.
You can change the default parameters in config.py

In [ ]:
cfg = config()

## Download the dataset

In [ ]:
dataset_dir = Path(cfg["dataset_dir"])

if not dataset_dir.exists():
    
    print("Download and extract dataset")
    # Get a single use download link from https://etsin.fairdata.fi/dataset/22b5191e-f24b-4457-93d3-95797c900fc0/data
    # You will only need "full images.zip" for the reidentification, generate a link to that.
    dataset_url = ""
    
    print(f'Creating directory "{dataset_dir}"')
    dataset_dir.mkdir(parents=True, exist_ok=True)
    
    file = wget.download(dataset_url.replace(" ", "%20"), out=str(dataset_dir))
    print()
    print(f'Extracting "{file}"')
    zip_f = zipfile.ZipFile(file, 'r')
    zip_f.extractall(dataset_dir)
    zip_f.close()
    Path(file).unlink()

print(f'SealID dataset is ready')

## Create dataset variables

In [ ]:
dataset_dir = Path(cfg["dataset_dir"])/"full images"

database_dataset = COCOImageDataset(dataset_dir/"source_database", dataset_dir/"annotation.csv", "database")
query_dataset = COCOImageDataset(dataset_dir/"source_query", dataset_dir/"annotation.csv", "query")


segmented_database_dataset = COCOImageDataset(dataset_dir/"segmented_database", dataset_dir/"annotation.csv", "database")
segmented_query_dataset = COCOImageDataset(dataset_dir/"segmented_query", dataset_dir/"annotation.csv", "query")

img_data = query_dataset[3]
print(img_data)

## Show an image from the dataset

In [ ]:
img, label = img_data

print_heading("Input image")
plt.imshow(img)
plt.show()

## Apply tonemapping

In [ ]:
# Remember to install pfstmo package with
# sudo apt-get install pfstmo

tonemapped_img = tonemap(img)

print_heading("Tonemapped image")
plt.imshow(tonemapped_img)
plt.show()

## Segment and crop an image

In [ ]:
segment_step = curry(segment, cfg["detectron_predictor"], instance_segmentation=False)

segmented_img, label = apply_pipeline(tonemapped_img, label, [segment_step, crop_step])[0]

print_heading("Segmented image")
plt.imshow(segmented_img)
plt.show()

## Extract pattern from an image

In [ ]:
extract_pattern_step = curry(extract_pattern, model=cfg["unet"])

pattern_img, label = apply_pipeline(segmented_img, label, [extract_pattern_step])[0]

print_heading("Pattern image")
plt.imshow(pattern_img)
plt.show()

## Encode an image

In [ ]:
encoded_img = encode_single(pattern_img, cfg)
print("Encoded image")

## Create a small database for testing
This example uses a very simple database implemented as a Python class. The database might take a lot of space if all images are used, and in that case it is better to store it on a disc, e.g. with the help of SQL based database systems.

In [ ]:
database_dataset_mini = DatasetSlice(database_dataset, range(20))

pipeline = [
            tonemap_step, 
            segment_step, crop_step,
            extract_pattern_step
            ]

pattern_dataset = apply_pipeline_cocodataset(database_dataset_mini, pipeline)

db_components, codebooks = create_database(pattern_dataset, cfg)

simple_db = SimpleDatabase(*db_components)
print("Created a test database")

## Perform re-identification

In [ ]:
identification_result = identify([(encoded_img, label)], database=simple_db, cfg=cfg)

## Visualise re-identification results

In [ ]:
visualise_match(identification_result[0])
print()

## Apply full pipeline to another query image

In [ ]:
full_pipeline = [
            tonemap_step, 
            segment_step, crop_step,
            extract_pattern_step,
            curry(encode_pipeline, cfg=cfg),
            curry(identify_single, database=simple_db, cfg=cfg),
            visualise_match
            ]

apply_pipeline(*query_dataset[0], full_pipeline)
print()